# Causal masks in vanilla transformer for text .. makes model autogressive

# vision model:
models that extract data from images.

input1 : image
input2 : prompt -"Where is the photographer resting?"

In [1]:
# Causal masks in vanilla transformer for text .. makes model autogressive

In [2]:
image -> contrastive vision encoder -> linear projection ->embeddingImage

prompt -> tokenizer( sentencePiece) ->embedddingPrompt

=> concatenated embedding (embeddingImage + embedddingPrompt) -> transformer decoder -> response

SyntaxError: invalid syntax (3227613456.py, line 1)

Each input image will be split into  blocks  of pixels creating a grid; then each block will be converted into an embedding. This embedding is a vector of fixed size.. that will be concenated with tokens' embeddings of  input prompt text i.e. each token will have an embedding just as each block of pixels will have an embedding. then this concatenated vetor will be sent to transformer decoder.

![alt text](vision_lang_model_01.png)


## Image encoder

### Contrastive learning

![alt text](vision_lang_model_02.png)

---


**Image encoder is vision transformer**

each I1 is in the embedding of image 1


**so for n image In with n descriptions Tn, we will get n*n matrix**



**we want dot prduct of image It with corresponding description text Tt to give higher value**
**and dot product of non corrspnding image and description should be low**
***so, we want diagonal to have high values and all non-diagonal low.***

### what is contrastive learninig?
so with contrastive learning we take list of images and a list of corresponding texts; we encoder them and then we want to train text encoer and image encoder to produce embeddings in such a way that dot product of image with its corresponding text is done, it should produce high value and the dot product of image with non-corresponding text should retunr lower value


**What is contrastive learning?**  
Contrastive learning is a machine learning technique where we work with pairs of related data, such as images and their corresponding text descriptions. The goal is to train two encoders—a text encoder and an image encoder—so that they generate embeddings (numerical representations) with specific properties:  
- The dot product of an image embedding and its corresponding text embedding should be high, indicating strong similarity.  
- Conversely, the dot product of an image embedding with a non-corresponding text embedding should be low, indicating dissimilarity.  

This approach helps the model learn to associate related data while distinguishing unrelated pairs effectively.  

**all non corresponding images are -ve samples**

Here’s a more technical and detailed explanation:  

**What is Contrastive Learning?**  
Contrastive learning is a representation learning paradigm that trains models by distinguishing between similar and dissimilar data pairs. It is widely used in tasks where the goal is to map data from different modalities (e.g., images and text) into a shared embedding space.  

In the context of image-text contrastive learning, the process involves:  

1. **Input Data**:  
   - A dataset consisting of pairs of images \( I \) and their corresponding textual descriptions \( T \).  
   - The dataset also implicitly includes negative pairs, where an image \( I \) is matched with a non-corresponding text \( T' \).  

2. **Encoders**:  
   - An **image encoder** \( f_I(I) \): Maps images into a high-dimensional embedding space. This is often a convolutional neural network (e.g., ResNet, Vision Transformer).  
   - A **text encoder** \( f_T(T) \): Maps text descriptions into the same embedding space. This is often a Transformer-based model (e.g., BERT, RoBERTa).  

3. **Objective**:  
   - The goal is to learn embeddings \( \mathbf{z}_I = f_I(I) \) for images and \( \mathbf{z}_T = f_T(T) \) for text such that:  
     - The **similarity score** (e.g., dot product or cosine similarity) between embeddings of corresponding pairs \( (\mathbf{z}_I, \mathbf{z}_T) \) is maximized.  
     - The similarity score between embeddings of non-corresponding pairs \( (\mathbf{z}_I, \mathbf{z}_{T'}) \) is minimized.  

4. **Loss Function**:  
   - A popular loss for contrastive learning is the **InfoNCE loss** (based on Noise Contrastive Estimation):  
     \[
     \mathcal{L} = - \frac{1}{N} \sum_{i=1}^{N} \left[ \log \frac{\exp(\text{sim}(\mathbf{z}_{I_i}, \mathbf{z}_{T_i}) / \tau)}{\sum_{j=1}^{N} \exp(\text{sim}(\mathbf{z}_{I_i}, \mathbf{z}_{T_j}) / \tau)} \right]
     \]  
     Here:  
     - \( \text{sim}(\mathbf{z}_{I}, \mathbf{z}_{T}) \) is the similarity measure (e.g., dot product or cosine similarity).  
     - \( \tau \) is a temperature hyperparameter that controls the sharpness of the distribution.  
     - \( N \) is the batch size.  
     - The numerator represents the similarity of the positive pair, while the denominator sums over similarities for all pairs in the batch (positive and negative).  

5. **Training Dynamics**:  
   - The encoders are trained jointly to minimize the contrastive loss. This ensures that embeddings of positive pairs are pulled closer together in the shared embedding space, while embeddings of negative pairs are pushed farther apart.  

6. **Applications**:  
   - Contrastive learning is foundational in models like **CLIP** (Contrastive Language-Image Pretraining), where it is used to align visual and textual modalities.  
   - It is also used in self-supervised learning frameworks (e.g., SimCLR, MoCo) to learn representations without explicit labels by treating augmentations of the same image as positive pairs and different images as negative pairs.  

This approach is highly effective for multimodal tasks, enabling downstream applications like image-text retrieval, zero-shot classification, and multimodal embedding alignment.  

Let me know if you’d like even more depth on any specific part!

### WE do this using cross entropy loss in contrastive laerning 


Using CEloss we can force a number(true label ) to have larger value. we will be cosidering vertical for text and horizontql for images

isn't CEloss just like a look up table where only true label value is considered? Remember


### Cross-Entropy Loss in Contrastive Learning  

In contrastive learning, **Cross-Entropy Loss (CE Loss)** is often employed to enforce alignment between corresponding pairs (e.g., images and texts) and separation between non-corresponding pairs. Here's a detailed explanation:

---

### **How Cross-Entropy Loss Works in Contrastive Learning**  

1. **Embedding Alignment**:  
   - We have embeddings for **images** and **texts**. For simplicity:  
     - Let \( \mathbf{z}_I \) represent the embedding of an image.  
     - Let \( \mathbf{z}_T \) represent the embedding of a text.  
   - The similarity between an image-text pair is computed, often using the **dot product** or **cosine similarity**.

2. **Similarity Matrix**:  
   - For a batch of \( N \) image-text pairs, we calculate the similarity scores for all pairs, forming a similarity matrix \( S \in \mathbb{R}^{N \times N} \):  
     \[
     S[i, j] = \text{sim}(\mathbf{z}_{I_i}, \mathbf{z}_{T_j})
     \]  
     - The \( i^{th} \) row corresponds to similarities between the \( i^{th} \) image and all texts in the batch.  
     - The \( j^{th} \) column corresponds to similarities between the \( j^{th} \) text and all images in the batch.

3. **Cross-Entropy Loss Objective**:  
   - Cross-Entropy Loss forces the model to focus on the **true labels** by maximizing the similarity score of the corresponding pair (positive pair) while minimizing the similarity scores for non-corresponding pairs (negative pairs).  
   - The CE Loss for the image-to-text direction can be written as:  
     \[
     \mathcal{L}_{I \to T} = - \frac{1}{N} \sum_{i=1}^N \log \frac{\exp(S[i, i] / \tau)}{\sum_{j=1}^N \exp(S[i, j] / \tau)}
     \]  
     Similarly, for the text-to-image direction:  
     \[
     \mathcal{L}_{T \to I} = - \frac{1}{N} \sum_{j=1}^N \log \frac{\exp(S[j, j] / \tau)}{\sum_{i=1}^N \exp(S[i, j] / \tau)}
     \]  
     - \( S[i, i] \): Similarity of the positive pair (image \( i \) with text \( i \)).  
     - \( \tau \): Temperature parameter controlling sharpness.  
     - The numerator focuses on the true label (positive pair), while the denominator aggregates over all possible labels (negative pairs included).

4. **Loss Aggregation**:  
   - The total loss is a combination of both directions:  
     \[
     \mathcal{L} = \mathcal{L}_{I \to T} + \mathcal{L}_{T \to I}
     \]  

---

### **Is Cross-Entropy Loss Like a Lookup Table?**  

In a way, **Cross-Entropy Loss acts like a lookup table** because it explicitly focuses on the **true label's score** (the positive pair) while treating other scores as negatives. However, it's more sophisticated than a simple lookup table:  

1. **Focus on the True Label**:  
   - CE Loss forces the score of the true label (positive pair) to dominate the similarity matrix, making it larger than all other scores.  

2. **Gradient-Based Optimization**:  
   - Unlike a static lookup table, CE Loss generates gradients for both the positive pair (to increase its similarity) and all negative pairs (to decrease their similarity).  
   - This dynamic adjustment ensures that the embeddings are optimized in a shared space.  

3. **Batch-Wide Comparison**:  
   - CE Loss considers the entire batch for computing the loss, encouraging embeddings to learn relationships relative to all samples in the batch.  

---

### **"Vertical for Text and Horizontal for Images"**  

This refers to how we interpret the similarity matrix \( S \):  
- **Rows (horizontal)**: Each row corresponds to one image and its similarity with all texts.  
- **Columns (vertical)**: Each column corresponds to one text and its similarity with all images.  

For contrastive learning:  
- Each **row** is used to compute the image-to-text loss (\( \mathcal{L}_{I \to T} \)).  
- Each **column** is used to compute the text-to-image loss (\( \mathcal{L}_{T \to I} \)).  

---

### Summary  

Cross-Entropy Loss in contrastive learning isn't just a lookup table—it’s a dynamic mechanism that enforces alignment for positive pairs and separation for negative pairs across the entire batch. It uses gradients to optimize the embeddings iteratively, ensuring the model learns meaningful relationships between modalities (e.g., images and text).

![alt text](vision_lang_model_03_code.png)

**labels = np.arrange(n) beccause of diagonal sequence i.e. 0,1,2,3...**

### ISSUES WITH CLIP 
we are using cross entropy loss and CE loss basically does comparsion between two distributions so we are converting each colunm/row into a distribution in n*n matrix. Conversion to distribution is done by softmax functon. Softwamx is numerically unstable.




![alt text](vision_lang_model_04_softmax.png)

***EXPONENTIAL fUNCTION GROWS TOO FAST AND MAY NOT FIT IN 32-BIT FLOATING POIN PRECISION***

### Issues with CLIP /softamax

CLIP (Contrastive Language–Image Pretraining) is a powerful multimodal model, but like any machine learning approach, it has limitations. A significant factor contributing to these issues stems from the use of **Cross-Entropy Loss (CE Loss)** in its contrastive learning framework. Here’s a breakdown of the challenges:

---

### **1. Cross-Entropy Loss and Distribution Comparison**  

- **Nature of CE Loss**:  
  Cross-Entropy Loss compares two probability distributions:  
  - One is the true label distribution (usually one-hot encoded).  
  - The other is the predicted distribution, obtained by normalizing similarity scores (e.g., using softmax).  
  This means CE Loss emphasizes maximizing the similarity of the true pair relative to the batch but may overlook absolute similarity.  

- **Impact on CLIP**:  
  - **Relative Comparisons**: CE Loss only ensures that positive pairs are more similar than negative pairs *within the batch*. It doesn’t guarantee high absolute similarity for the positive pairs.  
  - **Batch Dependence**: The performance of CLIP depends on the quality and diversity of negative samples in the batch. Poorly chosen negatives can lead to suboptimal training.  

---

### **2. Sensitivity to Batch Size**  

- Contrastive learning frameworks like CLIP are highly sensitive to batch size because the denominator in CE Loss involves all negative samples in the batch.  
- **Small Batch Size**:  
  - Reduces the diversity of negative samples.  
  - Leads to overfitting, where the model struggles to generalize beyond the batch.  
- **Large Batch Size**:  
  - Requires significant memory and computational resources.  
  - Makes training more expensive, especially for high-dimensional embeddings like those in CLIP.

---

### **3. Modality Gaps**  

- **Embedding Misalignment**:  
  CLIP aligns embeddings from two modalities (image and text) in a shared space. However, the distributions of embeddings for images and texts may not align perfectly due to differences in their inherent structures.  
  - Images have spatial and visual patterns.  
  - Text has sequential and semantic patterns.  
  This mismatch can lead to suboptimal performance in downstream tasks.  

- **Bias in Pretraining**:  
  The pretraining dataset and loss may inadvertently favor one modality (e.g., text) over the other, leading to less robust representations for the disadvantaged modality.  

---

### **4. Lack of Fine-Grained Supervision**  

- CLIP relies on global alignment between image and text embeddings. However, it does not explicitly enforce fine-grained relationships (e.g., parts of an image corresponding to specific words in the text).  
- This limitation can cause issues in tasks requiring precise alignment, such as object localization or detailed image-caption matching.

---

### **5. Dependence on Temperature Parameter (\( \tau \))**  

- The temperature parameter \( \tau \) in the softmax function controls the sharpness of the predicted probability distribution.  
  - A **small \( \tau \)** makes the model focus heavily on the most similar pair, potentially ignoring other relevant information.  
  - A **large \( \tau \)** spreads the focus, which can reduce contrast between positive and negative pairs.  
- Choosing the optimal \( \tau \) is non-trivial and can significantly affect performance.

---

### **6. Bias in Pretraining Data**  

- **Imbalanced Dataset**:  
  CLIP is trained on large-scale datasets scraped from the internet. These datasets may have biases (e.g., cultural, geographic, or demographic) that are inadvertently learned by the model.  
- **Noise in Data**:  
  The image-text pairs in web-scraped datasets can be noisy or irrelevant, leading to suboptimal learning.

---

### **7. Over-Reliance on Negative Samples**  

- CE Loss in contrastive learning depends heavily on negative samples to push embeddings apart. However:  
  - **Hard Negatives** (difficult non-corresponding pairs) are rare but crucial for training. Without them, the model might converge to a suboptimal solution.  
  - Overemphasis on hard negatives can lead to instability during training.

---

### **Summary of Key Issues**  

| **Issue**                  | **Impact**                                                                                     |  
|----------------------------|-----------------------------------------------------------------------------------------------|  
| CE Loss (relative comparisons) | Focuses on relative similarities, ignoring absolute similarity for positive pairs.          |  
| Sensitivity to Batch Size   | Small batches reduce diversity; large batches are computationally expensive.                 |  
| Modality Gaps               | Misalignment between image and text embeddings.                                              |  
| Lack of Fine-Grained Supervision | Cannot capture detailed relationships between image regions and text tokens.                |  
| Dependence on Temperature   | Suboptimal \( \tau \) can lead to poor alignment of embeddings.                               |  
| Dataset Bias and Noise      | Biases in web-scraped data can limit the generalizability of the model.                       |  
| Negative Sample Dependence  | Limited hard negatives can reduce training effectiveness; overemphasis causes instability.   |  

Addressing these issues often involves designing better loss functions (e.g., debiased contrastive loss), improving data quality, and incorporating additional supervision (e.g., fine-grained attention mechanisms). Let me know if you'd like elaboration on potential solutions!

### ISSUES WITH CLIP  

CLIP (Contrastive Language–Image Pretraining) faces certain challenges, one of which stems from its use of **Cross-Entropy Loss (CE Loss)** in its contrastive learning framework. Let’s delve into these issues:  

---

### **1. Cross-Entropy Loss and Distribution Comparison**  

- **Nature of CE Loss**:  
  Cross-Entropy Loss is designed to compare two probability distributions. In CLIP, it works on the **n × n similarity matrix**, where:  
  - Each row corresponds to a specific image or text.  
  - Each column corresponds to a distribution over all potential matches in the batch.  

- **How CE Loss Works in CLIP**:  
  - Each similarity score in the matrix is converted into a probability distribution using the **softmax function**.  
  - The model is trained to maximize the probability of correct (positive) pairs while minimizing the probability of incorrect (negative) pairs.  

---

### **2. Softmax Conversion Issues**  

The **softmax function** is a key component in converting raw similarity scores into probabilities, but it introduces several challenges:  

- **Numerical Instability**:  
  - Softmax involves exponentiating similarity scores, which can cause overflow or underflow when the values are very large or very small.  
  - This instability can lead to unreliable gradients, especially when the similarity scores in the matrix vary significantly.  

- **Exaggeration of Differences**:  
  - Softmax amplifies differences between similarity scores.  
  - This can cause the model to over-focus on the highest similarity score, potentially ignoring meaningful relationships between other pairs.  

---

### **3. Conversion to Row/Column Distributions**  

- In the **n × n similarity matrix**, rows represent images and columns represent texts (or vice versa).  
- Softmax is applied to each row (for images) or column (for texts) to convert raw scores into distributions.  
- **Limitations**:  
  - The process forces each row/column to sum to 1, but this does not inherently ensure meaningful alignment across modalities.  
  - It creates a dependency on the relative differences within the batch, which can degrade performance if the batch contains poor-quality negatives.

---

### **4. Dependence on Batch Quality**  

- **Small Batches**:  
  - Reduce the diversity of negative samples.  
  - Make the softmax normalization less effective because of limited contrast in similarity scores.  

- **Noisy Negatives**:  
  - In real-world datasets, some negative samples may not be truly irrelevant (e.g., an image and text might share subtle semantic similarities).  
  - These noisy negatives can confuse the model, reducing the effectiveness of CE Loss.  

---

### **5. Temperature Scaling in Softmax**  

- The softmax function in CLIP uses a **temperature parameter (\( \tau \))** to control the sharpness of the probability distribution:  
  - **Small \( \tau \)**: Focuses heavily on the highest similarity score, ignoring other scores.  
  - **Large \( \tau \)**: Produces a more uniform distribution, reducing contrast between positive and negative pairs.  
- Finding the optimal \( \tau \) is critical but challenging. Suboptimal temperature scaling can degrade model performance.

---

### **Key Observations**  

- The use of **softmax** in CE Loss enables contrastive learning but comes with trade-offs:  
  - It introduces **numerical instability**, especially with high-dimensional embeddings and diverse datasets.  
  - The focus on relative differences (via softmax normalization) may not capture absolute alignment effectively.  
- Addressing these issues may involve alternative loss functions (e.g., debiased contrastive loss) or improved numerical techniques (e.g., log-sum-exp trick to stabilize softmax).  

Let me know if you'd like further technical elaboration or examples!

### ISSUES WITH CLIP

CLIP (Contrastive Language–Image Pretraining) uses **Cross-Entropy Loss (CE Loss)** for contrastive learning, which involves comparing two distributions. The core issue here lies in the conversion of similarity scores into distributions using the **softmax function**, which can lead to **numerical instability** and precision issues, especially when dealing with large datasets and high-dimensional embeddings. Let's dive deeper into the specifics of this issue:

---

### **1. Cross-Entropy Loss and Distribution Comparison**

- **CE Loss Overview**:  
  Cross-Entropy Loss compares two probability distributions:  
  - One distribution is the true label (often one-hot encoded).  
  - The other is the predicted distribution, which is generated by applying the **softmax function** to similarity scores between images and text in the **n × n matrix**.

- **n × n Matrix**:  
  - Each row corresponds to a specific image or text (depending on whether you're comparing image-to-text or text-to-image).  
  - Each column represents a distribution over all potential matches in the batch (i.e., similarity scores with other images/texts).  

---

### **2. Numerical Instability with Softmax**

- **Softmax Function**:  
  The **softmax function** converts raw similarity scores (which can range from negative to positive) into probabilities by applying the exponential function to each similarity score, followed by normalization:
  
  \[
  P(x_i) = \frac{e^{x_i}}{\sum_{j} e^{x_j}}
  \]

  Where \(x_i\) is the similarity score for a specific image-text pair, and the denominator is the sum of exponentiated scores across all items in the batch.

- **Exponential Growth Issue**:  
  The **exponential function** grows very rapidly. When the similarity scores \(x_i\) are large (either positive or negative), applying the exponential function causes them to become very large or very small, which can lead to **overflow** or **underflow** during computation. This is especially problematic when the model works with high-dimensional data, such as image and text embeddings.

- **Precision Problems**:  
  In practice, floating-point precision (e.g., 32-bit floating-point) cannot handle extremely large or small numbers without loss of precision. This issue becomes particularly noticeable when:  
  - **Large values** (e.g., similarity scores of 100 or higher) are exponentiated, resulting in values too large to fit within the available precision.  
  - **Small values** (e.g., negative similarity scores leading to exponentiation of very small numbers) may cause underflow, resulting in values that are effectively zero.  

  This instability can cause incorrect gradients during backpropagation, leading to poor convergence or divergence in training.

---

### **3. Softmax Sensitivity and Precision Loss**

- **Effect of Exponential Growth**:  
  - The **exponential function** makes large similarity values (whether positive or negative) disproportionately dominant.  
  - As a result, even if a positive image-text pair has a moderate similarity, it may be overshadowed by a large negative or positive value, distorting the distribution.

- **Precision in 32-bit Floats**:  
  - **32-bit floating point** numbers have a limited range (approximately \(\pm 3.4 \times 10^{38}\)).  
  - Exponentiating large numbers can easily result in values that exceed this range, causing an overflow. Similarly, small negative values may underflow to zero, making them indistinguishable from each other.

---

### **4. Potential Solutions and Mitigations**

- **Log-Sum-Exp Trick**:  
  One way to mitigate this issue is to use the **log-sum-exp trick**, which stabilizes the computation of the softmax function by factoring out the largest value in the similarity scores before exponentiation. This reduces the range of values being exponentiated, preventing overflow or underflow:

  \[
  \text{softmax}(x) = \frac{e^{x - \max(x)}}{\sum_j e^{x_j - \max(x)}}
  \]

  This trick ensures that the exponential values are within a numerically stable range.

- **Clipping Large Values**:  
  Another approach is to clip extremely large similarity scores before applying softmax. By setting a maximum threshold for the similarity scores, we can avoid values that are too large to fit in the precision range.

- **Switch to 64-bit Precision**:  
  While this comes at the cost of increased memory usage, using **64-bit floating-point precision** (double precision) can alleviate the overflow/underflow problem by providing a much larger numerical range. However, this solution may not be feasible for large-scale models due to the computational overhead.

- **Use Alternative Loss Functions**:  
  Exploring alternative loss functions, such as **BCE (Binary Cross-Entropy)** or **triplet loss**, might offer more stable training without relying heavily on softmax normalization.

---

### **5. Summary of Key Issues**

| **Issue**                     | **Impact**                                                                                         |  
|-------------------------------|---------------------------------------------------------------------------------------------------|  
| **Exponential Growth in Softmax** | The exponential function grows too fast, causing numerical instability.                          |  
| **Precision Loss (32-bit Floats)** | Large or small values can lead to overflow or underflow, leading to poor gradient propagation.    |  
| **Effect on Training**         | Numerical instability can cause incorrect gradients and slow or unstable convergence during training. |

---

Addressing these issues requires careful handling of the numerical properties of the model. Using techniques like the **log-sum-exp trick** or switching to higher precision floating points can help mitigate these problems. Let me know if you'd like further details or clarification!

You've highlighted some important numerical stability issues with the cross-entropy loss implementation in CLIP (Contrastive Language-Image Pre-training). Let me break down the key points and discuss potential solutions:

1. Numerical Instability of Softmax
   - The softmax function involves exponentiating values, which can quickly lead to extremely large numbers
   - In 32-bit floating-point precision, this can cause overflow, leading to computational errors

2. Exponential Function Limitations
   - Exponential function grows extremely rapidly
   - When input values are large, exp(x) can exceed the maximum representable float32 value
   - This can cause:
     - Infinity values
     - Loss of precision
     - Potential model training instabilities

Potential Solutions:

1. Log-Sum-Exp Trick
```python
   def stable_softmax(x):
      # Subtract max value to prevent overflow
      max_x = np.max(x, axis=-1, keepdims=True)
      exp_x = np.exp(x - max_x)
      return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
```

2. Numerical Stabilization Techniques
   - Clip input values to a reasonable range
   - Use log-space computations
   - Implement careful scaling of input values

3. Consider using float64 precision if computational resources allow

4. Alternative Loss Formulations
   - Investigate modified contrastive loss approaches that avoid direct softmax computation

Would you like me to elaborate on any of these approaches or discuss implementation strategies for improving numerical stability?

# That's an excellent question. The scaling techniques I mentioned actually don't fundamentally change the model's performance when implemented correctly. Here's why:

1. Normalization, Not Alteration
   - The log-sum-exp trick and similar stabilization methods don't change the mathematical properties of the softmax
   - They simply prevent numerical overflow while maintaining the same relative probability distributions
   - The core computational goal remains unchanged

2. Preservation of Relative Relationships
   - By subtracting the max value before exponentation, you're preserving the relative differences between values
   - The softmax still produces the same ranking and proportional probabilities
   - Only the raw magnitude is adjusted to fit within computational limits

Example to illustrate:

```python
# Original problematic input
x = np.array([1000, 2000, 3000])

# Unstable softmax might produce inf or nan
unstable_softmax = np.exp(x) / np.sum(np.exp(x))

# Stabilized version
max_x = np.max(x)
stable_softmax = np.exp(x - max_x) / np.sum(np.exp(x - max_x))
```

In this example, the stabilized version produces a valid probability distribution without changing the fundamental comparative relationships between the original values.

Key Benefits:
- Prevents computational errors
- Maintains model logic
- Ensures consistent training behavior
- Allows precise gradient computation

The technique is essentially a mathematical "normalization" that keeps the core computational logic intact while preventing numerical instability.

Would you like me to elaborate on how this preservation of relative relationships works mathematically?

### **larger x resulting in infinity, so we are subtracting by max(x)**

In [ ]:
import numpy as np

# Original problematic input
x = np.array([10, 2000, 3000])

# Unstable softmax might produce inf or nan
unstable_softmax = np.exp(x) / np.sum(np.exp(x))
print(f" ----")
# Stabilized version
max_x = np.max(x)
stable_softmax = np.exp(x - max_x) / np.sum(
    np.exp(x - max_x)
)  # just subtract max(x) from x before np.exp()
unstable_softmax, stable_softmax

 ----


/tmp/ipykernel_117518/2405016546.py:6: RuntimeWarning: overflow encountered in exp
  unstable_softmax = np.exp(x) / np.sum(np.exp(x))
/tmp/ipykernel_117518/2405016546.py:6: RuntimeWarning: invalid value encountered in divide
  unstable_softmax = np.exp(x) / np.sum(np.exp(x))


(array([ 0., nan, nan]), array([0., 0., 1.]))

### range of logits

In general, **logits** in a **Cross-Entropy Loss** function can take any real value, and their range is theoretically **\((- \infty, + \infty)\)**.  

### Why Logits Can Be Unbounded:
- **Logits** are the raw, unnormalized scores produced by a model before applying the **softmax function**.  
- The **softmax** converts these logits into a probability distribution, but the logits themselves are not constrained.  

---

### Mathematical Context

For a classification problem:
1. **Logits**: \( z_i \) (output of the model for class \( i \)) can be any real number:  
   \[
   z_i \in (-\infty, +\infty)
   \]
2. **Softmax**: Converts the logits into probabilities:  
   \[
   p_i = \frac{e^{z_i}}{\sum_{j} e^{z_j}}
   \]
   - If \( z_i \) is very large, \( e^{z_i} \) dominates the numerator.  
   - If \( z_i \) is very small (negative), \( e^{z_i} \) approaches zero.

3. **Cross-Entropy Loss**:  
   Cross-Entropy Loss compares the predicted probabilities \( p_i \) with the true labels \( y_i \):  
   \[
   L = - \sum_{i=1}^C y_i \log(p_i)
   \]
   Here, the softmax ensures \( p_i \) is in the range \([0, 1]\), but the raw logits \( z_i \) are unconstrained.

---

### Range of Logits in Practice:
1. **Neural Networks**: Logits depend on the output of the last layer of the network. For fully connected layers:
   - No activation function is applied after the last layer.
   - Therefore, logits can be very large (positive or negative), especially if weights or inputs have large magnitudes.

2. **Stability of Softmax**:
   - Large positive logits (\( z_i \to +\infty \)) lead to probabilities close to \( 1 \).  
   - Large negative logits (\( z_i \to -\infty \)) lead to probabilities close to \( 0 \).  
   - This causes numerical instability due to the exponential growth of \( e^{z_i} \).

---

### Key Points:
- **Range of Logits**: \( (-\infty, +\infty) \).  
- **Range After Softmax**: \( (0, 1) \) (probabilities).  
- Large logits can cause **numerical instability** when exponentiated in the softmax function, particularly in low-precision floating-point formats (e.g., 32-bit).

---

### Mitigation Techniques:
1. **Logits Normalization**: Normalize logits before applying softmax.  
2. **Log-Sum-Exp Trick**: Stabilizes softmax computation by subtracting the maximum logit.  
   \[
   p_i = \frac{e^{z_i - \max(z)}}{\sum_{j} e^{z_j - \max(z)}}
   \]
3. **Regularization**: Apply techniques like weight decay to prevent very large weights, which could produce large logits.

In summary, logits are unbounded by design and can take any real value \((- \infty, + \infty)\).

The uploaded image provides a clear explanation of the **numerical instability of the softmax function** and a solution to mitigate it. Here’s a detailed breakdown of the content:

---

### **Softmax Function**  
The softmax function converts a vector of logits \( a_i \) into probabilities \( S_i \) such that:
\[
S_i = \frac{e^{a_i}}{\sum_{k=1}^N e^{a_k}}
\]
- \( a_i \) are the logits (raw scores) from the model.
- \( S_i \) is the probability for the \( i \)-th class.
- The softmax ensures \( S_i \in [0, 1] \) and \( \sum_{i} S_i = 1 \).

---

### **Problem: Numerical Instability**  
The softmax function involves the **exponential** \( e^{a_i} \), which grows very quickly for large \( a_i \):
- If \( a_i \) is very large, \( e^{a_i} \) can **overflow** and exceed the limits of 32-bit floating-point numbers.  
- If \( a_i \) is very small (negative), \( e^{a_i} \) becomes very close to zero, which can cause **underflow**.

This instability can cause the softmax computation to fail or produce inaccurate results.

---

### **Solution: Log-Sum-Exp Trick**  
To stabilize the softmax computation, we subtract the **maximum logit** \( \max_i (a_i) \) from all logits before applying the exponential:
\[
S_i = \frac{e^{a_i - \max_i (a_i)}}{\sum_{k=1}^N e^{a_k - \max_i (a_i)}}
\]
- By subtracting \( \max_i (a_i) \), the largest logit becomes \( 0 \), and all other logits are shifted to negative values.  
- This avoids numerical overflow because \( e^0 = 1 \) and the remaining terms \( e^{a_i - \max_i (a_i)} \) are in a manageable range.

---

### **Explanation in the Image**  
1. **Problem** (Red Text):  
   The exponential function grows too fast and may not fit in 32-bit floating-point precision.  

2. **Solution** (Green Text):  
   By subtracting the maximum logit, the arguments to the exponential function are pushed towards **negative values**, making the exponential outputs smaller and stable.

3. **Mathematical Derivation**:
   - The image derives the stabilized softmax step-by-step using a constant \( c \) where \( \log(c) = -\max_i (a_i) \).

---

### **Key Takeaways**:
- Softmax is numerically unstable because of the exponential growth of \( e^{a_i} \).  
- Stabilization is achieved using the **log-sum-exp trick** by subtracting the maximum logit.  
- This ensures that the computation is stable and avoids overflow/underflow issues.

Let me know if you'd like further clarification or examples! 🚀

### SIGNLIP

In siglip paper due to asymmetry of softmax loss, the normalization is independently performed two times; across images and across texts and matrix n*n is not symmetric because (1,2) is not same as (2,1)..***SO CLIP IS VERY COMPUTATIONALLY EXPENSIVE***    


So in SIGLIP, they have proposed to replace CE loss with Sigmoid, after computing n*n matrix(dot products), rather that treating loss as distribution over row or column, we treat it as a binary classification task using sigmoid. In  which each of these dot product is traeated independenly i.e. only diagoal values should be 1 we can do this using sigoid. it will take dot product as input, and sigmoid will return 1 for corresponding text and image, hence all of these dot products become indepedent

![alt text](vision_lang_model_05_siglip_softmax.png)

## **there are n labels(y_ij) and remember softmax is just normalizing probailities ... other than that we are only cindering value against label for both normalizations...so we are not actually conisdering rows/colums for loss calculations but just normalization.**

and then we apply log for each softmax

![alt text](vision_lang_model_02_CEloss.webp)

In the **SigLIP** paper (Scaling the Learning of Image-Text Pretraining), the authors address the **asymmetry** of the standard softmax loss in contrastive learning setups, particularly in methods like CLIP. Here’s an elaboration:

---

### **Issue with Softmax Loss in Contrastive Learning**  
In a typical **contrastive learning setup** (e.g., CLIP), the loss uses a single softmax normalization across either:
1. **Rows** (image-to-text matching) or  
2. **Columns** (text-to-image matching)  

This creates an **asymmetry** because the softmax loss is only applied in one direction at a time:
- If the loss normalizes across rows, it aligns each **image embedding** to the corresponding **text embedding**.
- If the loss normalizes across columns, it aligns each **text embedding** to the corresponding **image embedding**.

However, this **single softmax normalization** does not treat images and texts symmetrically, leading to **imbalanced training dynamics**.

---

### **SigLIP's Solution: Dual Softmax Normalization**  
To address this asymmetry, **SigLIP independently normalizes across both images and texts**. The loss is computed **twice**, once for images and once for texts:
1. **Image-to-Text Loss**:  
   Normalize the logits (dot products) **row-wise** to match each image embedding with its corresponding text embedding.  
2. **Text-to-Image Loss**:  
   Normalize the logits **column-wise** to match each text embedding with its corresponding image embedding.

The total loss is then the **average of both losses**.

---

### **Mathematical Formulation**  
Let \( z_i \) be the embedding of image \( i \), \( t_j \) be the embedding of text \( j \), and \( \tau \) be the temperature parameter.

The **dual softmax contrastive loss** is:

\[
\mathcal{L} = \frac{1}{2} \left( \mathcal{L}_{\text{img-to-text}} + \mathcal{L}_{\text{text-to-img}} \right)
\]

Where:  
- **Image-to-Text Loss**:
\[
\mathcal{L}_{\text{img-to-text}} = -\frac{1}{N} \sum_{i=1}^N \log \frac{\exp(z_i \cdot t_i / \tau)}{\sum_{j=1}^N \exp(z_i \cdot t_j / \tau)}
\]

- **Text-to-Image Loss**:
\[
\mathcal{L}_{\text{text-to-img}} = -\frac{1}{N} \sum_{j=1}^N \log \frac{\exp(t_j \cdot z_j / \tau)}{\sum_{i=1}^N \exp(t_j \cdot z_i / \tau)}
\]

Here:  
- \( z_i \cdot t_j \) is the dot product (similarity) between image \( i \) and text \( j \).  
- The **row-wise softmax** applies to the image embeddings, and the **column-wise softmax** applies to the text embeddings.  

---

### **Benefits of Dual Normalization**  
1. **Symmetry**: Treats images and texts equally during training, ensuring balanced learning.  
2. **Stability**: Independent normalization avoids potential instability caused by combining image and text distributions into a single softmax.  
3. **Improved Performance**: Experiments in SigLIP show better alignment of embeddings and improved downstream task performance.

---

### **Summary**  
- In **SigLIP**, the asymmetry of the softmax loss is resolved by performing **dual normalization**.  
- Softmax is applied **independently** across rows (images) and columns (texts).  
- This results in a symmetric, balanced loss that improves the training dynamics of image-text contrastive learning models.

Let me know if you’d like me to expand on any part! 🚀

# SIGLIP REPLACES SOFTMAX with SIGMOID



So in SIGLIP, they have proposed to replace CE loss with Sigmoid, after computing n*n matrix(dot products), rather that treating loss as distribution over row or column, we treat it as a binary classification task using sigmoid. In  which each of these dot product is traeated independenly i.e. only diagoal values should be 1 we can do this using sigoid. it will take dot product as input, and sigmoid will return 1 for corresponding text and image, hence all of these dot products become indepedent



INDEPENDENT because as their is no normalization, we can spit whole matrics into block and each block can be compiuted on different devices.

![alt text](vision_lang_model_06_siglip_sigmoid.png)

You are correct! In **SigLIP**, the authors propose to replace the **Cross-Entropy (CE) loss** with a **Sigmoid-based Binary Cross-Entropy (BCE) loss** for contrastive learning. This modification simplifies the loss calculation by treating the problem as a **binary classification task** rather than a multi-class distribution task. Let’s break this down:

---

### **Problem with Cross-Entropy (CE) Loss**
1. In standard contrastive learning (e.g., CLIP), CE loss works with a **softmax normalization** over rows or columns of the **N × N similarity matrix** (where \( N \) is the batch size).  
2. This normalization forces the logits (dot products) to behave like **probability distributions**:
   - Row-wise softmax aligns **image-to-text** pairs.
   - Column-wise softmax aligns **text-to-image** pairs.  
3. However, softmax introduces issues like:  
   - **Numerical instability** due to the exponential function (softmax).  
   - **Asymmetry** in loss calculation (softmax over rows vs. columns).  
   - Tight coupling between dot products in the matrix (non-diagonal values influence the normalization).

---

### **Sigmoid-based Binary Cross-Entropy (BCE) Loss**
Instead of treating the dot products as part of a single probability distribution, SigLIP treats each dot product **independently** as a **binary classification task**.

#### Key Idea:
- Each entry \( s_{ij} \) in the **N × N similarity matrix** (dot product between image \( i \) and text \( j \)) is treated as an **independent prediction**.
- The goal is to classify:
  - **Diagonal entries** (\( i = j \)) as **positive pairs** (label = 1).  
  - **Off-diagonal entries** (\( i \neq j \)) as **negative pairs** (label = 0).  

#### **Sigmoid Function**:
The sigmoid function maps each dot product \( s_{ij} \) into the range \( (0, 1) \), where:
\[
\text{Sigmoid}(s_{ij}) = \frac{1}{1 + e^{-s_{ij}}}
\]
- \( \text{Sigmoid}(s_{ij}) \approx 1 \) when \( s_{ij} \) is large (high similarity for positive pairs).  
- \( \text{Sigmoid}(s_{ij}) \approx 0 \) when \( s_{ij} \) is small (low similarity for negative pairs).  

#### **Binary Cross-Entropy (BCE) Loss**:
The BCE loss for the \( N \times N \) similarity matrix can be written as:
\[
\mathcal{L} = - \frac{1}{N^2} \sum_{i=1}^N \sum_{j=1}^N \left[ y_{ij} \log(\sigma(s_{ij})) + (1 - y_{ij}) \log(1 - \sigma(s_{ij})) \right]
\]
Where:
- \( \sigma(s_{ij}) \) is the sigmoid of the dot product \( s_{ij} \).  
- \( y_{ij} = 1 \) for diagonal entries (positive pairs).  
- \( y_{ij} = 0 \) for off-diagonal entries (negative pairs).  

---

### **Why is This Better?**
1. **Independence of Dot Products**:  
   Each dot product \( s_{ij} \) is treated **independently** of others. There is no normalization over rows or columns, removing interdependencies caused by softmax.  

2. **Numerical Stability**:  
   Sigmoid is more numerically stable than softmax because it avoids the exponential growth caused by softmax normalization.  

3. **Simpler Loss**:  
   The loss directly focuses on ensuring that diagonal entries (correct image-text pairs) are **highly similar** and off-diagonal entries (incorrect pairs) are **dissimilar**.  

4. **Symmetry**:  
   The BCE loss inherently treats images and texts symmetrically, unlike the asymmetric softmax loss.

---

### **Summary**  
In SigLIP:  
1. The **similarity matrix** (dot products) is computed as \( N \times N \).  
2. The task is reframed as a **binary classification problem**:  
   - Diagonal values (positive pairs) should be **1**.  
   - Off-diagonal values (negative pairs) should be **0**.  
3. **Sigmoid** is applied to each dot product independently, followed by the **Binary Cross-Entropy loss**.  
4. This eliminates softmax’s numerical instability and asymmetry, making the training more stable and robust.

This change leads to better numerical behavior, improved performance, and a cleaner formulation of the loss function.

Let me know if you’d like further clarifications or examples! 🚀

### Can we use CE for two classes 
Yes, you can use **cross-entropy loss** instead of **binary cross-entropy (BCE)** for 2 classes, but it depends on how you format your labels and outputs.

### Understanding the Difference:
1. **Binary Cross-Entropy (BCE)**:
   - Used for **binary classification** where labels are **0 or 1**.
   - The model typically outputs a **single probability** (from the sigmoid function) for the positive class, and the negative class probability is \( 1 - \text{probability} \).

2. **Cross-Entropy Loss** (Categorical Cross-Entropy):
   - Used for **multi-class classification**, where labels are one-hot encoded, and the model outputs probabilities for each class (via **softmax**).
   - For 2 classes, the softmax output is still valid, but it produces two probabilities, one for each class.

---

### Can We Use Cross-Entropy Loss for 2 Classes?
**Yes**, cross-entropy loss can be used for binary classification **if you treat the problem as a 2-class multi-class classification**. Here's how it works:

- Instead of using **BCE with sigmoid**, you:
   - Use a **softmax function** at the output layer (which produces probabilities for 2 classes).
   - Use **cross-entropy loss** with the 2-class outputs.

---

### Key Differences Between BCE and Cross-Entropy for 2 Classes:
| Aspect                     | BCE (Binary Cross-Entropy)               | Cross-Entropy for 2 Classes         |
|----------------------------|-----------------------------------------|-------------------------------------|
| **Output Layer**           | Sigmoid (single output, 0–1)            | Softmax (two outputs summing to 1)  |
| **Labels**                 | Single value (0 or 1)                   | One-hot encoded vector [1,0] or [0,1] |
| **Loss Function**          | BCE (binary)                            | Cross-Entropy (categorical)         |
| **Model Output**           | Single probability (positive class)     | Two probabilities (for both classes) |

---

### When to Use Which:
- **Use BCE**: If your model outputs a **single probability** (via sigmoid) for the positive class.
- **Use Cross-Entropy**: If your model outputs **two probabilities** (via softmax) for the two classes.

---

### Practical Note:
- Mathematically, **binary cross-entropy** and **cross-entropy for 2 classes** produce the **same loss value** when implemented correctly. The difference is how you represent the outputs and labels.

For example:
- **BCE**:
   - Output: \( p \) (probability of class 1).
   - Label: \( y \) (0 or 1).
   - Loss: \( - [ y \log(p) + (1-y) \log(1-p) ] \).

- **Cross-Entropy**:
   - Output: \( [p_1, p_2] \) where \( p_1 + p_2 = 1 \).
   - Label: \( [1,0] \) or \( [0,1] \).
   - Loss: \( - [ y_1 \log(p_1) + y_2 \log(p_2) ] \).

Both are equivalent for 2 classes, just reformulated.

---

### Why Libraries Combine Them:
Many frameworks like **PyTorch** provide a single loss function, such as:
- `BCEWithLogitsLoss`: Combines sigmoid + BCE for binary classification.
- `CrossEntropyLoss`: Combines softmax + cross-entropy for multi-class classification, including the 2-class case.

In summary, while BCE is more efficient for binary classification, cross-entropy loss can still be used for 2 classes with a softmax output.

    BCE returns single value for 2 claasses so sigmoid gives label 0 or 1

    for two claaes, CE will return 2 values [0,1]

#### ***our vision encoder is contrastive but not our vision language model***

# CONTRASTIVE VISION ENCODER -- VISION TRANSFORMER

## WHY CONTRASTIVE VISION ENCODER? why not ordinary vision encoder?
In Sigclip, we want this embedding to not only capture infomration about image but can be contrasted or can be used along with text embedings and this is exactly we do in vision embedding model.

and contrasted vision encoder is easy to train because its just crawl it from internet.

### ONLY OUR VISION ENCODER model IS CONTRaASTED
![alt text](vision_lang_model_01.png)

Exactly! In **SigLIP** (and other contrastive learning frameworks like CLIP), the goal is to ensure that the **vision embeddings** and **text embeddings** share a common **representation space**. This shared space allows embeddings from the two modalities (images and text) to be **comparable** and **contrasted** effectively.

---

### **Key Idea**: Shared Representation Space
The embeddings are trained so that:
- An **image embedding** (from the vision model) aligns closely with its **corresponding text embedding** (from the text model).
- Non-matching pairs (e.g., unrelated images and texts) are pushed apart in the embedding space.

---

### **Vision Embedding Model**  
In the vision model:
1. The input image is passed through a backbone neural network (e.g., ResNet, ViT - Vision Transformer) to produce a **fixed-dimensional embedding**.  
2. This embedding is **rich** in visual information and represents the semantic content of the image.  
3. The vision embeddings are not just generic visual features—they are explicitly trained to **align** with the corresponding text embeddings.

---

### **Why Is Contrast Important?**  
The ability to **contrast** embeddings across modalities (image and text) is critical because:
- It allows the model to **discriminate** between correct (positive) and incorrect (negative) image-text pairs.  
- This contrastive behavior is achieved by ensuring:
   - Positive pairs (correct image-text pairs) have **high similarity** in the embedding space.
   - Negative pairs (incorrect image-text pairs) have **low similarity**.

---

### **Embedding Alignment Process**  
The training process enforces this alignment using a **contrastive loss**:
1. **Dot Products**: The similarity between an image embedding \( v_i \) and a text embedding \( t_j \) is computed as their **dot product**:  
   \[
   s_{ij} = v_i \cdot t_j
   \]
2. **Sigmoid-based Binary Classification**: In **SigLIP**, each dot product \( s_{ij} \) is treated as a binary classification task:
   - Diagonal entries \( s_{ii} \) (positive pairs) are pushed toward 1 (high similarity).  
   - Off-diagonal entries \( s_{ij} \) (negative pairs) are pushed toward 0 (low similarity).  

This way, the embedding space becomes **jointly optimized** for both image and text inputs.

---

### **Outcome**: Joint Embedding Space
By training the vision embedding model to align with the text embeddings:
- The vision embeddings capture **not only visual features** but also **semantic information** that can be interpreted in relation to text.  
- This allows for tasks like **image-text retrieval**, **zero-shot classification**, and other multimodal applications.

---

### **Summary**  
In SigLIP (and CLIP-like models):
- The **vision embeddings** are designed to **align** with text embeddings in a **shared representation space**.  
- This alignment is achieved through **contrastive learning** with a loss function (e.g., sigmoid-based BCE loss).  
- The embeddings can then be **compared** and **contrasted** effectively across modalities, enabling multimodal tasks.

This makes the vision model not just a generic feature extractor but a model that understands images in a way that is **semantically compatible** with natural language. 🚀Exactly! In **SigLIP** (and other contrastive learning frameworks like CLIP), the goal is to ensure that the **vision embeddings** and **text embeddings** share a common **representation space**. This shared space allows embeddings from the two modalities (images and text) to be **comparable** and **contrasted** effectively.

---

### **Key Idea**: Shared Representation Space
The embeddings are trained so that:
- An **image embedding** (from the vision model) aligns closely with its **corresponding text embedding** (from the text model).
- Non-matching pairs (e.g., unrelated images and texts) are pushed apart in the embedding space.

---

### **Vision Embedding Model**  
In the vision model:
1. The input image is passed through a backbone neural network (e.g., ResNet, ViT - Vision Transformer) to produce a **fixed-dimensional embedding**.  
2. This embedding is **rich** in visual information and represents the semantic content of the image.  
3. The vision embeddings are not just generic visual features—they are explicitly trained to **align** with the corresponding text embeddings.

---

### **Why Is Contrast Important?**  
The ability to **contrast** embeddings across modalities (image and text) is critical because:
- It allows the model to **discriminate** between correct (positive) and incorrect (negative) image-text pairs.  
- This contrastive behavior is achieved by ensuring:
   - Positive pairs (correct image-text pairs) have **high similarity** in the embedding space.
   - Negative pairs (incorrect image-text pairs) have **low similarity**.

---

### **Embedding Alignment Process**  
The training process enforces this alignment using a **contrastive loss**:
1. **Dot Products**: The similarity between an image embedding \( v_i \) and a text embedding \( t_j \) is computed as their **dot product**:  
   \[
   s_{ij} = v_i \cdot t_j
   \]
2. **Sigmoid-based Binary Classification**: In **SigLIP**, each dot product \( s_{ij} \) is treated as a binary classification task:
   - Diagonal entries \( s_{ii} \) (positive pairs) are pushed toward 1 (high similarity).  
   - Off-diagonal entries \( s_{ij} \) (negative pairs) are pushed toward 0 (low similarity).  

This way, the embedding space becomes **jointly optimized** for both image and text inputs.

---

### **Outcome**: Joint Embedding Space
By training the vision embedding model to align with the text embeddings:
- The vision embeddings capture **not only visual features** but also **semantic information** that can be interpreted in relation to text.  
- This allows for tasks like **image-text retrieval**, **zero-shot classification**, and other multimodal applications.

---

### **Summary**  
In SigLIP (and CLIP-like models):
- The **vision embeddings** are designed to **align** with text embeddings in a **shared representation space**.  
- This alignment is achieved through **contrastive learning** with a loss function (e.g., sigmoid-based BCE loss).  
- The embeddings can then be **compared** and **contrasted** effectively across modalities, enabling multimodal tasks.

This makes the vision model not just a generic feature extractor but a model that understands images in a way that is **semantically compatible** with natural language. 🚀

### Vision transformer
takes in an image as split into pacthes i.e. 16 patches. then we extract information from each patch using convolution producing an embedding for each patch and then flatten all embeddings rsulting in loss of positional information, order doesn't matter here; we just concatenate them. basically we are loosing two dimensionality here.
Then we add positional encoding to this embedding vector to contains positional information. UNlike vanilla transformer, These positional encoding are not calculated using sinisidual function but they are learned so that position one in positional encoding vector always get added to top left patch:1 ; and 4 at top right.. so that model still has acces to 2d infomration even though data itelf is flattened. Model will run positional encdoing. Then we feed it to transformer. Transformer does contextualization of this embedding. so transformer intakes a series of embeddings each representing one single patch; the output of transformer through attention mechanism is a series of embeddings but each of these embeedings only not capturing infomration about itself but also about other patches.

In language models we use causal masks because text models contains infomration about previous words(autoregressive models) but we don't need that here in vision transformer.Becasue in image there is no auto regressiveness(no sequential order). SO these contextualized embddings not only capture information about themselves but also all other images. So we use these embeedings to capture info about each patch butalso how it is present in image. SO we want each patch to contain infomratoin about its position which is given y positional encoding but we are also concerned about patch's sorroundings in image by cointextualizing.




------------------------------------



![alt text](vision_lang_model_08_vit_contextualization.png)


---

![alt text](vision_lang_model_07_vitp_input.png)


### Vision Transformer (ViT) Explained

The **Vision Transformer (ViT)** adapts the **Transformer architecture**, originally designed for NLP, to process images. Here’s a detailed breakdown of the explanation you provided:

---

### 1. **Input Image as Patches**
- The input image is divided into **non-overlapping patches** (e.g., 16x16 pixels each).
- For a standard image of size \( 224 \times 224 \), splitting into 16x16 patches results in \( 14 \times 14 = 196 \) patches.
- Each patch is treated like a "token" in a Transformer, similar to how words are tokens in NLP models.

---

### 2. **Patch Embedding with Convolution**
- To extract features from each patch, we use a **convolution operation** or a **linear projection**.
- This produces a **vector embedding** for each patch, representing the features extracted from that patch.
- **Flattening**: These patch embeddings are flattened into a **1D sequence** (order doesn't matter yet, and we lose 2D spatial structure at this point).

---

### 3. **Loss of Positional Information**
- Flattening the embeddings removes the **2D positional information** (the spatial relationship between patches, like top-left or bottom-right).
- Without positional information, the Transformer would not "know" where each patch came from, which is crucial for images.

---

### 4. **Positional Encoding**
To restore the positional information:
- We add **positional encodings** to each patch embedding.  
- **Learned Positional Encoding**: Unlike vanilla Transformers (which use fixed sinusoidal functions), Vision Transformers **learn the positional encodings** during training.
    - For example, the position \( P_1 \) in the positional encoding vector will always get added to the **top-left patch**.
    - Position \( P_4 \) will correspond to the **top-right patch**, and so on.
- This ensures the model retains **2D spatial awareness** even though the embeddings are flattened into a sequence.

---

### 5. **Feeding Embeddings to the Transformer**
- The positional encodings are added to the patch embeddings to create **position-aware embeddings**.
- These embeddings are then fed into the **Transformer**.
- The Transformer operates on this sequence of embeddings and performs **self-attention**.

---

### 6. **Contextualization via Attention**
- In a Transformer, **self-attention** allows each patch embedding to interact with all other patch embeddings.
- The output of the Transformer is a new set of embeddings where:
  - Each embedding contains **information about itself** (features of the patch).
  - It also captures information about **other patches** (contextual information).

This is similar to how words in NLP models "attend" to each other to understand context, but here, patches attend to each other to understand the overall structure of the image.

---

### 7. **No Causal Masks in Vision Transformers**
- In language models, we often use **causal masks** for autoregressive training (e.g., predicting the next word while only looking at previous words).
- In Vision Transformers, there is **no sequential dependency** between patches like in text.  
- Images are not sequential, so we do **not** use causal masks.

Instead:
- Each patch embedding can "see" all other patches simultaneously through self-attention.
- This helps the model capture both **local** (features of a single patch) and **global** (relationship with other patches) information.

---

### 8. **Summary**
- The Vision Transformer splits an image into patches and converts each patch into an embedding.
- **Positional encodings** are added to restore spatial relationships (learned during training).
- The Transformer contextualizes the embeddings using self-attention, enabling each patch to capture:
   - Its **own features**.
   - Context from **surrounding patches**.
- This approach removes the need for convolutional layers or causal masks and allows the Transformer to process the image holistically.

---

### Why is This Effective?
- By using self-attention, the model can **globally model relationships** between patches, which is harder for traditional CNNs that operate with local receptive fields.
- Positional encodings ensure that spatial relationships are retained, even when the image is represented as a sequence.

Let me know if you’d like a visual representation or further clarifications on any part! 😊

### Vision Transformer (ViT): In-Depth Explanation

The **Vision Transformer (ViT)** is a model that applies the Transformer architecture, originally designed for NLP tasks, to computer vision. Instead of using convolutional neural networks (CNNs) to process images, ViT uses **self-attention** to capture both local and global features of an image. Let’s break it down in more depth:

---

## 1. **Image as Patches**

- **Traditional CNNs** process images as 2D grids and use convolutional filters to extract spatial features.  
- **ViT**, on the other hand, treats an image as a **sequence of patches**, much like words in a sentence for NLP.  
- The image \( I \) of size \( H \times W \times C \) (Height × Width × Channels) is divided into **non-overlapping patches** of size \( P \times P \), where \( P \) is the patch size.

### Example:
For an image of size \( 224 \times 224 \times 3 \) (standard ImageNet input):
- If \( P = 16 \), the image is divided into \( 14 \times 14 = 196 \) patches.
- Each patch has dimensions \( 16 \times 16 \times 3 \), which are **flattened** into a vector of size \( 16 \times 16 \times 3 = 768 \).

---

## 2. **Patch Embeddings**

Each patch is treated as a **token**. To convert these patches into a form usable by the Transformer:
- **Linear Projection**: A linear layer (or convolution) is applied to **flatten each patch** into a 1D embedding vector.  
- This linear layer maps each patch \( P \) (size \( P \times P \times C \)) into a **D-dimensional embedding vector**:
  \[
  x_i = \text{Linear}(\text{Flatten}(P_i))
  \]
  where \( x_i \) is the embedding for patch \( i \).

- This gives a sequence of embeddings:
  \[
  X = [x_1, x_2, ..., x_N] \quad \text{where} \quad N = \frac{H \times W}{P^2}
  \]
  \( N \) is the total number of patches.

---

## 3. **Positional Encodings**

The Transformer does not inherently understand the **order** or **position** of the patches.  
To provide positional information, **positional encodings** are added to the patch embeddings.

### Why is Positional Encoding Needed?
- After flattening the patches into a sequence, the **2D spatial structure** of the image is lost.
- Without positional information, the Transformer treats the patches as unordered tokens.

### Learned Positional Encoding:
- Unlike NLP Transformers that often use **sinusoidal positional encodings** (fixed), ViT **learns positional encodings** during training.  
- Each positional encoding \( PE_i \) is a learnable vector added to the patch embedding \( x_i \):
  \[
  z_i = x_i + PE_i
  \]
  where \( z_i \) is the position-aware embedding for patch \( i \).

### 2D Spatial Awareness:
- The positional encodings are learned such that:
  - The first positional encoding corresponds to the **top-left patch**.
  - The last positional encoding corresponds to the **bottom-right patch**.
- This ensures the model retains **2D spatial relationships** even though the patches are flattened.

---

## 4. **Class Token (Optional)**

In addition to the patch embeddings, ViT introduces a special **[CLS] token** (classification token) similar to BERT:
- A **learnable vector** \( x_{cls} \) is prepended to the sequence of patch embeddings.
- The output corresponding to this token at the final layer is used for **classification**.

### Input to the Transformer:
The input to the Transformer is the sequence:
\[
Z = [x_{cls}, z_1, z_2, ..., z_N]
\]
where \( z_i \) are the position-aware patch embeddings.

---

## 5. **Transformer Encoder**

The Transformer encoder processes the sequence of embeddings. It consists of:
1. **Multi-Head Self-Attention (MHSA)**:  
   - Allows each patch embedding to attend to all other patches.
   - Each patch captures **global context** by interacting with all other patches.

2. **Feed-Forward Network (FFN)**:  
   - A position-wise MLP (Multi-Layer Perceptron) applied to each embedding.

3. **Layer Normalization** and **Residual Connections**:  
   - Ensure stable training and efficient gradient flow.

---

## 6. **Self-Attention in ViT**

The self-attention mechanism is key to ViT. It allows patches to interact with each other:
- Each patch embedding \( z_i \) queries the other embeddings \( z_j \) using **Query, Key, and Value** projections:
  \[
  \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
  \]
  where:
  - \( Q = z_i W_Q \) (Query),
  - \( K = z_j W_K \) (Key),
  - \( V = z_j W_V \) (Value),
  - \( d_k \) is the dimensionality of the keys.

- The result is that each patch embedding is updated to include:
  - Its own information.
  - Contextual information from all other patches.

---

## 7. **No Causal Masking**

In NLP tasks, causal masks ensure that a token can only attend to previous tokens (autoregressive property).  
In ViT:
- **No causal masks** are used because patches do not have a sequential order like words in a sentence.
- Each patch can attend to all other patches **simultaneously**.

This allows the model to capture **global context** across the entire image.

---

## 8. **Output of the Transformer**

The output of the Transformer is a **sequence of contextualized embeddings**:
- Each embedding \( z_i' \) corresponds to a patch, but it now contains:
  - Information about the patch itself.
  - Information about the **relationships** between the patch and all other patches.

For classification:
- The output corresponding to the **[CLS] token** is used as the image representation.

For segmentation or reconstruction tasks:
- The outputs corresponding to each patch are used.

---

## 9. **Advantages of Vision Transformers**

1. **Global Context**:
   - Unlike CNNs, which rely on local receptive fields, ViT uses self-attention to model **global relationships** between patches.

2. **Flexibility**:
   - ViT is more flexible than CNNs because it does not impose a **fixed inductive bias** (like locality or translation invariance).

3. **Scalability**:
   - When trained on large datasets (e.g., ImageNet-21k or JFT-300M), ViTs outperform CNNs.

---

## 10. **Summary Workflow**

1. **Split the Image into Patches**: Convert \( H \times W \) image into \( N \) patches.
2. **Linear Projection**: Embed each patch into a \( D \)-dimensional vector.
3. **Add Positional Encodings**: Restore spatial information lost during flattening.
4. **Transformer Encoder**:
   - Process the sequence of embeddings using **self-attention**.
   - Capture global and contextual information.
5. **Output**:
   - Use the **[CLS] token** for classification or patch embeddings for other tasks.

---

### Key Insights:
- Vision Transformers treat images as sequences of patches and use self-attention to model relationships between patches.
- Positional encodings ensure that spatial information is preserved.
- The Transformer encoder captures both **local features** and **global context** without relying on convolution.

Would you like to dive deeper into specific parts, such as self-attention math or training strategies for ViT? 😊

# 01_modelling_siglip.py

In [2]:
from typing import Optional, Tuple
import torch
import torch.nn as nn

# poligemma


class SiglipVisionConfig:
    def __init__(
        self,
        hidden_size=768,  # embedding size
        intermediate_size=3072,  # size of linear layer
        num_hidden_layers=12,  # number of layers of vision transformer
        num_attention_heads=12,  # number of heads in multihead attention
        num_channels=3,  # RGB
        image_size=224,
        patch_size=14,
        layer_norm_eps=1e-6,
        attention_dropout=0.0,
        num_image_tokens: int = None,  # how many output embedding we will have for each image; each of these contextualized embedding will be considered as a tokens of image.It wont ba a one single embrding that represents whole imagebut list of embeddings that represesnt a patch of each image and als info about other patches throigh the attention mechanismo
        **kwargs
    ):
        super().__init__()

        self.hidden_size = hidden_size
        self.intermediate_size = (intermediate_size,)
        self.num_hidden_layers = (num_hidden_layers,)
        self.num_attention_heads = (num_attention_heads,)
        self.num_channels = (num_channels,)
        self.image_size = (image_size,)
        self.patch_size = (patch_size,)
        self.layer_norm_eps = (layer_norm_eps,)
        self.attention_dropout = (attention_dropout,)
        self.num_image_tokens = num_image_tokens

This Python code defines a **configuration class** for a custom Vision Transformer (ViT)-based architecture, called `SiglipVisionConfig`. This configuration class is intended to hold hyperparameters and settings that control the structure and behavior of a Vision Transformer model. Let’s break it down step by step:

---

### **1. Imports**
```python
from typing import Optional, Tuple
import torch
import torch.nn as nn
```
- **`typing`**: Provides support for type hints like `Optional` and `Tuple`.
- **`torch` and `torch.nn`**: Used for defining and implementing neural network layers and operations in PyTorch.

---

### **2. `SiglipVisionConfig` Class**
This class serves as a configuration container for the Vision Transformer model. The hyperparameters defined here control various aspects of the architecture, such as the number of layers, attention heads, and embedding sizes.

#### **Constructor (`__init__` Method)**
```python
class SiglipVisionConfig:
    def __init__(
        self,
        hidden_size=768,  # Size of the embedding vector for each patch
        intermediate_size=3072,  # Dimensionality of the MLP layers in the transformer
        num_hidden_layers=12,  # Number of transformer encoder layers
        num_attention_heads=12,  # Number of attention heads in multi-head self-attention
        num_channels=3,  # Number of image channels (e.g., 3 for RGB)
        image_size=224,  # Height/Width of the input image
        patch_size=14,  # Size of each patch (14x14 pixels)
        layer_norm_eps=1e-6,  # Epsilon value for LayerNorm (numerical stability)
        attention_dropout=0.0,  # Dropout rate for attention
        num_image_tokens: int = None,  # Number of contextualized embeddings (tokens) for the image
        **kwargs
    ):
        super().__init__()
```

The **parameters** define the key components of the Vision Transformer. Let’s dive into each:

#### **Key Parameters**
1. **`hidden_size=768`**:
   - The dimensionality of the embedding for each patch after projection.  
   - Each patch of the image will be represented by a vector of size \( 768 \).

2. **`intermediate_size=3072`**:
   - The size of the intermediate layer in the **feed-forward network (FFN)** inside each Transformer encoder block.  
   - Typically, \( \text{intermediate\_size} = 4 \times \text{hidden\_size} \).

3. **`num_hidden_layers=12`**:
   - Number of **Transformer encoder layers** (or blocks) in the model.

4. **`num_attention_heads=12`**:
   - Number of heads in **multi-head self-attention**.
   - Each attention head works independently and captures relationships between patches.

5. **`num_channels=3`**:
   - The number of input channels in the image (e.g., 3 for RGB, 1 for grayscale).

6. **`image_size=224`**:
   - The height and width of the input image (e.g., \( 224 \times 224 \)).

7. **`patch_size=14`**:
   - The size of each non-overlapping patch.  
   - If \( \text{image\_size} = 224 \) and \( \text{patch\_size} = 14 \), the image is divided into \( (224 / 14) \times (224 / 14) = 16 \times 16 = 256 \) patches.

8. **`layer_norm_eps=1e-6`**:
   - A small constant added to the denominator in **Layer Normalization** for numerical stability.

9. **`attention_dropout=0.0`**:
   - Dropout rate applied to the attention weights during training to prevent overfitting.

10. **`num_image_tokens`**:
    - Specifies the number of contextualized embeddings (tokens) for the image after processing by the Transformer.
    - Each embedding represents information about a specific patch while incorporating relationships with other patches through the attention mechanism.

    #### Key Insight:
    - Instead of representing the whole image as a single embedding, the model produces **multiple embeddings** (one for each patch). This allows for richer representations that preserve **spatial and contextual information**.

---

### **3. Additional Attributes**
After the parameters are passed into the constructor, they are stored as attributes of the class:
```python
self.hidden_size = hidden_size
self.intermediate_size = (intermediate_size,)
self.num_hidden_layers = (num_hidden_layers,)
self.num_attention_heads = (num_attention_heads,)
self.num_channels = (num_channels,)
self.image_size = (image_size,)
self.patch_size = (patch_size,)
self.layer_norm_eps = (layer_norm_eps,)
self.attention_dropout = (attention_dropout,)
self.num_image_tokens = num_image_tokens
```

- Each hyperparameter is assigned to an attribute (e.g., `self.hidden_size`), making it accessible later when defining or using the model.
- Some attributes are stored as **tuples** (e.g., `self.intermediate_size = (intermediate_size,)`), possibly for compatibility with other parts of the code.

---

### **4. Comments in the Code**
The comment under `num_image_tokens` provides important clarification:
```python
# how many output embeddings we will have for each image; 
# each of these contextualized embeddings will be considered as tokens of the image.
# It won’t be a single embedding that represents the whole image, 
# but a list of embeddings that represent a patch of the image and also info 
# about other patches through the attention mechanism.
```

This explains that:
- **`num_image_tokens`** determines how many embeddings (tokens) are produced for each image.  
- Instead of summarizing the entire image into one embedding (as in CNNs), ViT outputs a **sequence of embeddings**, one for each patch.  
- These embeddings are enriched by the **self-attention mechanism**, which allows each patch to encode information about other patches.

---

### **5. Purpose of the Class**
This class provides a centralized way to:
1. **Store hyperparameters** for the Vision Transformer model.
2. **Configure the architecture** dynamically by modifying the parameters.

When designing the model, these attributes (e.g., `hidden_size`, `num_attention_heads`) will be used to construct the layers (e.g., multi-head attention, feed-forward networks).

---

### **6. Next Steps**
- This class is only the configuration container. To implement the Vision Transformer:
  - Use these attributes to define layers like **patch embeddings**, **multi-head self-attention**, and **MLPs**.
  - Add training logic (e.g., classification head) on top of the transformer outputs.

Would you like to see how to use this configuration to build the actual Vision Transformer? 😊

In [ ]:
class SiglipVisionModel(nn.Module):

    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.config = config
        self.visio_model = SiglipVisionTransformer(config)

    def forward(self, pixel_values) -> Tuple:
        """
        [batch_size, num_channels, height, width] => [batch_size, num_image_tokens/num_patches, hidden_size/embedding_dim]
        takes in batch of images and returns list of embeddigs for each image in batch
        """

        return self.vision_model(pixel_values=pixel_values)

Not necessarily, but they are closely related. Let’s clarify the difference:

---

### **Number of Patches**
The **number of patches** refers to how the input image is divided into smaller non-overlapping segments. It is calculated as:

\[
\text{Number of patches} = \left(\frac{\text{image\_size}}{\text{patch\_size}}\right)^2
\]

For example:
- If the image size is \( 224 \times 224 \) and the patch size is \( 14 \times 14 \), then:
  \[
  \text{Number of patches} = \left(\frac{224}{14}\right)^2 = 16 \times 16 = 256
  \]

Each patch will then be converted into a **patch embedding** of size `hidden_size` (e.g., 768).

---

### **num_image_tokens**
The **`num_image_tokens`** refers to the number of output tokens produced by the Vision Transformer. Typically:
- If there are no additional tokens (like a class token), **`num_image_tokens` is equal to the number of patches**.
- However, if additional tokens (e.g., a classification token or other special tokens) are added, then:
  \[
  \text{num\_image\_tokens} = \text{number of patches} + \text{number of special tokens}
  \]

For example:
1. If there are **256 patches** and no extra tokens, then:
   \[
   \text{num\_image\_tokens} = 256
   \]

2. If there are **256 patches** and **1 class token** (as in the original Vision Transformer), then:
   \[
   \text{num\_image\_tokens} = 256 + 1 = 257
   \]

---

### **Key Differences**
| **Aspect**                | **Number of Patches**                      | **num_image_tokens**                            |
|---------------------------|--------------------------------------------|------------------------------------------------|
| **Definition**            | Number of patches extracted from the image | Number of output tokens after processing       |
| **Includes Class Token?** | No                                         | Sometimes (if class tokens are used)           |
| **Typical Usage**         | Input size for the Transformer             | Output size of the Transformer (per sequence) |

---

### **Conclusion**
- In most cases, **`num_image_tokens` is the same as the number of patches**, unless extra tokens (e.g., class tokens) are added to the input sequence.  
- If your Vision Transformer does not use a class token, then:
  \[
  \text{num\_image\_tokens} = \text{number of patches}
  \]
Would you like more details on how class tokens or special tokens are used in Vision Transformers? 😊

In [ ]:
class SiglipVisionTransformer(nn.Module):
    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.config = config
        embed_dim = config.hidden_size

        self.embeddings = SiglipVisionEmbeddings(
            config
        )  # pacthes will be converted to embeddings
        self.encoder = SiglipEncoder(config)
        self.post_layernorm = nn.LayerNorm(embed_dim, eps=config.layer_norm_eps)

    def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
        # pixel values: [batch_size, num_channels, height, width] => [batch_size, num_image_tokens/num_patches, hidden_size/embedding_dim]
        hidden_states = self.embeddings(pixel_values)
        last_hidden_state = self.encoder(input_embds=hidden_states)
        last_hidden_state = self.post_layernorm(last_hidden_state)
        return last_hidden_state

Let’s break this code down step by step:

---

### **1. Class Definition**
The `SiglipVisionTransformer` class is a custom implementation of a Vision Transformer (ViT). It uses two main components:
- **Embeddings**: Converts input image patches into embeddings.
- **Encoder**: Applies the Transformer architecture (multi-head self-attention and feedforward layers) to process these embeddings.

---

### **2. Constructor (`__init__`)**
```python
def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    embed_dim = config.hidden_size

    self.embeddings = SiglipVisionEmbeddings(config) # Patches will be converted to embeddings
    self.encoder = SiglipEncoder(config)
    self.post_layernorm = nn.LayerNorm(embed_dim, eps=config.layer_norm_eps)
```

- **`config`**: An instance of `SiglipVisionConfig` that holds all the hyperparameters (e.g., `hidden_size`, `num_attention_heads`, etc.).
- **`embed_dim`**: Equal to `hidden_size` (the size of the patch embeddings).
- **`self.embeddings`**: Responsible for:
  - Dividing the input image into patches.
  - Converting each patch into a fixed-size embedding.
  - Adding positional encodings to retain spatial information.
- **`self.encoder`**: The Transformer encoder, which applies self-attention and feedforward layers to contextualize the embeddings.
- **`self.post_layernorm`**: A Layer Normalization applied after the encoder to stabilize the output.

---

### **3. Forward Method (`forward`)**
The `forward` method defines how the input data flows through the model.

```python
def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
    # pixel_values: [batch_size, num_channels, height, width] => [batch_size, num_image_tokens/num_patches, hidden_size/embedding_dim]
    hidden_states = self.embeddings(pixel_values)
    last_hidden_state = self.encoder(input_embds = hidden_states)
    last_hidden_state = self.post_layernorm(last_hidden_state)
    return last_hidden_state
```

#### **Input**
- **`pixel_values`**: A batch of input images with shape:
  \[
  [\text{batch\_size}, \text{num\_channels}, \text{height}, \text{width}]
  \]

#### **Steps**
1. **Convert Image to Embeddings (`self.embeddings`)**:
   - The image is divided into patches.
   - Each patch is flattened and projected into an embedding of size `hidden_size`.
   - Positional encodings are added to these embeddings.
   - The output shape is:
     \[
     [\text{batch\_size}, \text{num\_patches}, \text{hidden\_size}]
     \]

2. **Contextualize Embeddings (`self.encoder`)**:
   - The embeddings are passed through the Transformer encoder.
   - Self-attention allows each patch to attend to information from all other patches.
   - The output shape remains:
     \[
     [\text{batch\_size}, \text{num\_patches}, \text{hidden\_size}]
     \]

3. **Post Layer Normalization (`self.post_layernorm`)**:
   - Layer normalization is applied to stabilize the outputs.

#### **Output**
- **`last_hidden_state`**:
  - A tensor containing the final contextualized embeddings for all patches.
  - Shape:
    \[
    [\text{batch\_size}, \text{num\_patches}, \text{hidden\_size}]
    \]

---

### **4. Key Points**
- **`self.embeddings`** handles patch extraction and positional encoding.
- **`self.encoder`** processes embeddings through multi-head attention and feedforward layers, capturing relationships between patches.
- **`self.post_layernorm`** ensures numerical stability and normalization after encoding.

---

### **High-Level Flow**
1. **Input**: Raw pixel values \([B, C, H, W]\).
2. **Embeddings**:
   - Divide the image into patches.
   - Convert each patch into a fixed-size embedding.
   - Add positional encodings.
   - Output shape: \([B, N, D]\), where \(N\) is the number of patches, and \(D\) is `hidden_size`.
3. **Transformer Encoder**:
   - Contextualize the embeddings using self-attention.
   - Output shape: \([B, N, D]\).
4. **Normalization**:
   - Apply LayerNorm to the final embeddings.
   - Output shape: \([B, N, D]\).

---

Would you like a detailed explanation of any specific part (e.g., embeddings, encoder, or attention mechanism)? 😊

### encoder 


afer converting to embedding, we'll run it  through list of transformwe encoder layers
In case of vision transformer.. normalization is done before FEED FORWARD and MULTI-HEAD ATTENTION
![alt text](vision_lang_model_09_vit_encoder_layers.png)